In [1]:
#########################################################
### ejemplo de generador de data de hidrógeno verde #####
### a partir de fuentes púlicas de datos ################
#########################################################

%%capture
!pip install pvlib

## William F. Holmgren, Clifford W. Hansen, and Mark A. Mikofski.
## “pvlib python: a python package for modeling solar energy systems.”
## Journal of Open Source Software, 3(29), 884, (2018).
## https://doi.org/10.21105/joss.00884

from google.colab import files
import json
import numpy as np
import matplotlib.pyplot as plt
from pvlib import location
from pvlib import irradiance
import pandas as pd
from matplotlib import pyplot as plt

# Building Q and getting penalties with a heuristic

In [2]:
import numpy as np
import random

def objective_function(x, Q):
    return np.dot(np.dot(x, Q), x)

def flip_bit(x, index):
    new_x = np.copy(x)
    new_x[index] = 1 - new_x[index]
    return new_x

def simulated_annealing(Q, max_iter=1000, initial_temp=100, cooling_rate=0.99):
    n = len(Q)
    x = np.random.randint(2, size=n)  # Initialize with a random binary vector
    current_energy = objective_function(x, Q)

    for i in range(max_iter):
        temp = initial_temp * (cooling_rate ** i)

        # Randomly select a bit to flip
        index_to_flip = random.randint(0, n-1)

        # Generate new candidate solution
        new_x = flip_bit(x, index_to_flip)
        new_energy = objective_function(new_x, Q)

        # Acceptance criteria
        if new_energy < current_energy:
            x = new_x
            current_energy = new_energy
        else:
            delta_energy = new_energy - current_energy
            if random.random() < np.exp(-delta_energy / temp):
                x = new_x
                current_energy = new_energy

    return x, current_energy


In [3]:
def get_Q(penalties, data):
  # Penalties
  Pmin1, Pmin2, Pmax1, Pmax2 = penalties

  ## Electricity price:
  c_g = data['grid_buying_price_values']

  # Batery:
  Cap_b = data['e_bat_capacity_values'][0]
  Watt_b = data['e_bat_charging_power_values'][0]
  Alpha = Watt_b/Cap_b

  SOC_0 = data['e_bat_SOC_ini']
  SOC_min = data['e_bat_SOC_min']
  SOC_max = data['e_bat_SOC_max']

  ## Matrix A.
  A = np.zeros((12, 12))

  A[0][0] = 2*Watt_b*c_g[0]/3
  A[1][1] = 4*Watt_b*c_g[0]/3

  ## Matrix B.
  B = np.zeros((12, 12))

  B[2][2] = 2*Watt_b*c_g[1]/3
  B[3][3] = 4*Watt_b*c_g[1]/3

  ## Matriz C.

  C = np.zeros((12, 12))
  beta_c = SOC_min - SOC_0 + Alpha

  C[0][0] = Pmin1*4/9**Alpha**2 -4/3*beta_c*Alpha
  C[1][1] = Pmin1*16/9**Alpha**2 -8/3*beta_c*Alpha
  C[4][5] = Pmin1*4/9
  C[4][0] = Pmin1*C[5][0] - 4/9*Alpha
  C[4][1] = - Pmin1*8/9*Alpha
  C[0][1] = Pmin1*16/9*Alpha**2
  C[4][4] = Pmin1*1/9 + Pmin1*2*beta_c/3
  C[5][5] = Pmin1*4/9 + Pmin1*4*beta_c/3

  ## Matrix D.

  D = np.zeros((12, 12))

  beta_d = SOC_min - SOC_0 + 2*Alpha

  D[0][0] = D[2][2] = Pmin2*4/3*Alpha*(Alpha/3 - beta_d)
  D[1][1] = D[3][3] = Pmin2*8/3*Alpha*(2*Alpha/3 - beta_d)

  D[6][6] = Pmin2*1/9 + Pmin2*2*beta_d/3
  D[7][7] = Pmin2*4/9 + Pmin2*4*beta_d/3

  D[0][1] = D[0][3] = D[1][2] = D[2][3] = Pmin2*16/9*Alpha**2
  D[1][3] = Pmin2*32/9*Alpha**2
  D[0][2] = Pmin2*8/9*Alpha**2

  D[3][7] = D[1][7] = - Pmin2*16/9*Alpha
  D[0][6] = D[2][6] = - Pmin2*4/9*Alpha
  D[0][7] = D[1][6] = D[2][7] = D[3][6] = - Pmin2*8/9*Alpha

  D[6][7] = Pmin2*4/9

  ## Matrix E.

  E = np.zeros((12, 12))

  beta_e = SOC_0 - SOC_max - Alpha

  E[0][0] = Pmax1*4/9**Alpha**2 + 4/3*beta_c*Alpha
  E[1][1] = Pmax1*16/9**Alpha**2 + 8/3*beta_c*Alpha
  E[8][8] = Pmax1*1/9 + Pmax1*2*beta_c/3
  E[9][9] = Pmax1*4/9 + Pmax1*4*beta_c/3
  E[0][1] = Pmax1*16/9*Alpha**2
  E[0][8] = Pmax1*4/9*Alpha
  E[8][9] = Pmax1*4/9
  E[0][9] = E[1][8] = Pmax1*8/9*Alpha
  E[1][9] = Pmax1*16/9*Alpha

  ## MatriX F.

  F = np.zeros((12, 12))

  beta_f = SOC_0 - 2*Alpha - SOC_max

  F[0][0] = F[2][2] = Pmax2*4/3*Alpha*(Alpha/3 + beta_f)
  F[1][1] = F[3][3] = Pmax2*8/3*Alpha*(2*Alpha/3 + beta_f)
  F[10][10] = Pmax2*1/9 + Pmax2*2*beta_f/3
  F[11][11] = Pmax2*4/9 + Pmax2*4*beta_f/3

  F[0][1] = F[0][3] = F[1][2] = F[2][3] = Pmax2*16/9*Alpha**2
  F[0][2] = Pmax2*8/9*Alpha**2
  F[1][3] = Pmax2*32/9*Alpha**2

  F[0][10] = F[2][10] = Pmax2*4/9*Alpha
  F[0][11] = F[1][10] = F[2][11] = F[3][10] = Pmax2*8/9*Alpha
  F[3][11] = F[1][11] = Pmax2*16/9*Alpha

  F[10][11] = Pmax2*4/9

  Q = A + B + C + D + E + F

  return Q

In [ ]:
# Small test with non representative values
data_test = {"e_bat_SOC_ini": 0.5, "e_bat_SOC_min": 0.1, "e_bat_SOC_max": 0.9, 'e_bat_charging_power_values': [24*3.5], "e_bat_capacity_values":24*[5], 'grid_buying_price_values':[123.5, 112.4]}

Q_test = get_Q(penalties=[1,2,3,4], data=data_test)
Q_test

In [ ]:
# Run simulated annealing

# Define the Q matrix (3x3 as an example)
#Q = np.array([[1, -1, 0],
#             [-1, 1, 0],
#            [0, 0, 0]])

x, energy = simulated_annealing(Q_test)
print(f"Optimal solution: x = {x}, energy = {energy}")

Optimal solution: x = [0 0 0 0 0 0 0 0 0 0 1 1], energy = -268.0


In [4]:
####Testing all posible combinations
# def get_labels(data):
#   # We give more weight to the min values because we would like the battery to operate closer to the maximum because of performance issues of batteries when they operate near their minimum charge.
#   Pmin1, Pmax1 = 500, 100
#   Pmin2_values = [Pmin1 + i for i in np.arange(0, 400, 45)] # 10 each
#   Pmax2_values = [Pmax1 + i for i in np.arange(0, 500, 60)]

#   best_combination = (Pmin1, Pmin2_values[0], Pmax1, Pmax2_values[0])
#   minimum = 10000000000
#   for pmin2 in Pmin2_values:
#     for pmax2 in Pmax2_values:
#       penalties = [Pmin1, pmin2, Pmax1, pmax2]
#       Q = get_Q(penalties, data)
#       x, minimum_energy = simulated_annealing(Q)
#       if minimum_energy < minimum:
#         best_combination = penalties
#         minimum = minimum_energy

#   print(f"best_combination: {best_combination}, minimum temp: {minimum}")
#   return [float(i) for i in best_combination]

###Equal increments
def get_labels(data):
  # We give more weight to the min values because we would like the battery to operate closer to the maximum because of performance issues of batteries when they operate near their minimum charge.
  Pmin1, Pmax1 = 500*10**6, 100*10**6
  pmin2, pmax2 = 300*10**6, 60*10**6
  best_combination = (Pmin1, pmin2, Pmax1, pmax2)
  minimum = 10**21
  INCREMENT = 2*10**7
  for i in range(40):
    pmin2 += INCREMENT
    pmax2 += INCREMENT
    penalties = [Pmin1, pmin2, Pmax1, pmax2]
    Q = get_Q(penalties, data)
    x, minimum_energy = simulated_annealing(Q)
    if minimum_energy < minimum:
      best_combination = penalties
      minimum = minimum_energy

  print(f"best_combination: {best_combination}, minimum temp: {minimum}")
  return [float(i) for i in best_combination]

In [ ]:
%time
get_labels(data_test)

# Building the dataset for penalties estimation.
### The dataset contains the following columns/inputs:
- Battery capacity (MWh)
- Maximum battery power (MW)
- Electricity price range (euros/MWh)
- Electrolyzer maximum power (MW)
- H2 demand (tons per time instant)
- Irradiation (W/m^2)

Besides, the Q matrix will be stored temporarily in order to execute the penalty finding heuristic.

### And the target columns are the four penalties.




In [5]:
def get_irradiance(site_location, date, tilt, surface_azimuth, timesteps):
    # Creates one day's worth of 10 min intervals
    times = pd.date_range(date, freq='240min', periods=timesteps,
                          tz=site_location.tz)
    # Generate clearsky data using the Ineichen model, which is the default
    # The get_clearsky method returns a dataframe with values for GHI, DNI,
    # and DHI
    clearsky = site_location.get_clearsky(times)
    # Get solar azimuth and zenith to pass to the transposition function
    solar_position = site_location.get_solarposition(times=times)
    # Use the get_total_irradiance function to transpose the GHI to POA
    POA_irradiance = irradiance.get_total_irradiance(
        surface_tilt=tilt,
        surface_azimuth=surface_azimuth,
        dni=clearsky['dni'],
        ghi=clearsky['ghi'],
        dhi=clearsky['dhi'],
        solar_zenith=solar_position['apparent_zenith'],
        solar_azimuth=solar_position['azimuth'])
    # Return DataFrame with only GHI and POA
    return pd.DataFrame({'GHI': clearsky['ghi'],
                         'POA': POA_irradiance['poa_global']})

def generate_data_entry(file_sample_number, soc_ini=0.5, capacity_H2_storage = 20, max_cover_irradiation = 1.0, range_volatility_grid_price = 0.2, demand_H2 = 30, max_power_battery = 3.5, battery_capacity = 5, season="summer"):
  data = {}

  data['metadata'] = {}
  data['metadata']['date'] = '23/08/2023'

  ## All units in hour format
  data['metadata']['w1'] = 'All units are in hour time format, to reduce the timestep granularity, the scale should be changed'

  ###########################
  ### Número de timesteps ###
  ###########################

  N = 2

  data['timesteps'] = N
  ############
  ### GRID ###
  ############

  # Electricity buying price. Source OMIE. €/Mwh. https://www.omie.es/es/spot-hoy
  data['grid_buying_price_units'] = '€/Mwh'

  # Manually sampled data from source
  data['grid_buying_price_values'] = [151.84, 132.22]

  ## Se generan muestras a partir de los datos
  volatilidad = np.random.uniform(1 - range_volatility_grid_price, 1 + range_volatility_grid_price, N)

  data['grid_buying_price_values'] = [a*b for a,b in zip(data['grid_buying_price_values'], volatilidad)]

  # plt.plot(data['grid_buying_price_values'])
  # plt.title("electricity buying prices")

  # PV generation

  # Puertollano
  tz = 'Etc/GMT+2'
  lat, lon = 38.6871200, -4.1073400

  site = location.Location(lat, lon, tz=tz)

  # Dos días cualesquiera, con elevación de 35º y orientados casi hacia el sur (180º) 150º
  summer_irradiance = get_irradiance(site, '06-20-2023 08:00:00', 64, 150, N)
  winter_irradiance = get_irradiance(site, '12-21-2023 10:00:00', 25, 140, N)

  # Convert Dataframe Indexes to Hour:Minute format to make plotting easier
  summer_irradiance.index = summer_irradiance.index.strftime("%H:%M")
  winter_irradiance.index = winter_irradiance.index.strftime("%H:%M")

  # Plot GHI vs. POA for winter and summer
  # fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
  # summer_irradiance['GHI'].plot(ax=ax1, label='GHI')
  # summer_irradiance['POA'].plot(ax=ax1, label='POA')
  # winter_irradiance['GHI'].plot(ax=ax2, label='GHI')
  # winter_irradiance['POA'].plot(ax=ax2, label='POA')
  # ax1.set_xlabel('Time of day (Summer)')
  # ax2.set_xlabel('Time of day (Winter)')
  # ax1.set_ylabel('Irradiance ($W/m^2$)')
  # ax1.legend()
  # ax2.legend()
  # plt.show()

  # Asumimos la producción proporcional (en MW) a la irrandiancia sobre el POA
  # junto con una cobertura nubosa aleatoria

  cobertura = np.random.uniform(0.5, max_cover_irradiation, N)

  effective_meters = 30000 # Engloba el rendimiento de los módulos y del inversor

  data['gen_PV_units'] = 'MW'
  #data['gen_PV_values'] = [0, 0, 0, 0, 0, 0, 0, 0.1, 0.13, 0.16, 0.20, 0.34, 0.52,
  #                          0.67, 0.79, 0.7, 0.5, 0.3, 0.18, 0.1, 0, 0, 0, 0]

  # summer_irradiance['POA'] is en W/m^2, we go to MW multiplying by m^2 and dividing by 10^6
  irradiance = summer_irradiance if season == "summer" else winter_irradiance

  data['gen_PV_values'] = [effective_meters*a*b/1000000 for a,b in zip(irradiance['POA'], cobertura)]

  # plt.plot(data['gen_PV_values'])
  # plt.title("gen_PV")

  ########################
  ### Almac. Eléctrico ###
  ########################

  # Capacidad batería

  data['e_bat_capacity_units'] = 'MWh'
  data['e_bat_capacity_values'] = N*[float(battery_capacity)]

  # SOC inicial, SOC_min y SOC_max

  data['e_bat_SOC_ini'] = soc_ini
  data['e_bat_SOC_min'] = 0.1
  data['e_bat_SOC_max'] = 0.9
  # Pot. máxima carga/descarga

  data['e_bat_charging_power_units'] = 'MW'
  data['e_bat_charging_power_values'] = N*[float(max_power_battery)]

  ######################
  ### Electrolizador ###
  ######################

  # Potencia máxima electrolizador

  data['electrolyzer_max_power_units'] = 'MW'
  data['electrolyzer_max_power_values'] = N*[50]

  # Rendimiento e/H2, 39.4 kWh/kg, tipically close to 70%
  # https://www.homerenergy.com/products/pro/docs/latest/electrolyzer_efficiency.html -- no anda


  data['electrolyzer_performance_units'] = 'tH2/MWh'
  data['electrolyzer_performance_values'] = N*[1/56.3]

  ##################
  ### Demanda H2 ###
  ##################

  data['H2_demand_units'] = 'ton de H2'
  data['H2_demand_values'] = N*[float(demand_H2)]

  #################
  ### Almac. H2 ###
  #################

  # Capacidad Almacén

  data['H2_storage_SOC_capacity_units'] = 'tons H2'
  data['H2_storage_SOC_capacity_values'] = N*[float(capacity_H2_storage)]

  # SOC inicial Almacén

  data['H2_storage_SOC_ini'] = 0.5
  data['H2_storage_SOC_min'] = 0.1
  data['H2_storage_SOC_max'] = 0.9

  ########################################
  ########### Dato estrella ##############
  ########### THE Q MATRIX  ##############
  ########################################
  penalties = get_labels(data) #  A list with the penalties, in this case, only four.
  data["labels"] = penalties

  json.dump(data, open(f'dataset_reduced/json_H2_start2_{file_sample_number}.json', 'w'))
  print(file_sample_number)

In [ ]:
# Test with one file having default values
generate_data_entry(file_sample_number="test", season="winter")

best_combination: [500, 840, 100, 470], minimum temp: -1714.2136786530657
{'metadata': {'date': '23/08/2023', 'w1': 'All units are in hour time format, to reduce the timestep granularity, the scale should be changed'}, 'timesteps': 2, 'grid_buying_price_units': '€/Mwh', 'grid_buying_price_values': [160.29807849246404, 153.1846139105907], 'gen_PV_units': 'MW', 'gen_PV_values': [12.919289713496923, 1.808682326047676], 'e_bat_capacity_units': 'MWh', 'e_bat_capacity_values': [5.0, 5.0], 'e_bat_SOC_ini': 0.5, 'e_bat_SOC_min': 0.1, 'e_bat_SOC_max': 0.9, 'e_bat_charging_power_units': 'MW', 'e_bat_charging_power_values': [3.5, 3.5], 'electrolyzer_max_power_units': 'MW', 'electrolyzer_max_power_values': [50, 50], 'electrolyzer_performance': {}, 'electrolyzer_performance_units': 'tH2/MWh', 'electrolyzer_performance_values': [0.017761989342806397, 0.017761989342806397], 'H2_demand_units': 'ton de H2', 'H2_demand_values': [30.0, 30.0], 'H2_storage_SOC_capacity_units': 'tons H2', 'H2_storage_SOC_ca

## Bulk data generation

In [12]:
!rm -rf dataset_reduced
!mkdir dataset_reduced

In [13]:
import threading

%time
sample_amount = 0

amount_already_produced = 0

threads = []
for i in np.arange(0.1, 2, 0.1):
  #grid price volatility
  if sample_amount > amount_already_produced:
    thread = threading.Thread(target=generate_data_entry, kwargs={"file_sample_number":f"{sample_amount}_electricity", "range_volatility_grid_price":i})
    threads.append(thread)
    sample_amount+=1

  #Battery capacity
  for i in np.arange(3, 15, 1):
    if sample_amount > amount_already_produced:
      thread = threading.Thread(target=generate_data_entry, kwargs={"file_sample_number":f"{sample_amount}_battery", "battery_capacity":i})
      threads.append(thread)
    sample_amount+=1

    #Battery power
    for i in np.arange(2, 5, 0.2):
      if sample_amount > amount_already_produced:
        thread = threading.Thread(target=generate_data_entry, kwargs={"file_sample_number":f"{sample_amount}_battery_power", "max_power_battery":i})
        threads.append(thread)
      sample_amount+=1

      #Battery initial state of charge
      for i in np.arange(0.5, 0.9, 0.1):
        if sample_amount > amount_already_produced:
          thread = threading.Thread(target=generate_data_entry, kwargs={"file_sample_number":f"{sample_amount}_season", "soc_ini":i})
          threads.append(thread)
        sample_amount+=1


   # for i in np.arange(0.5, 0.8, 0.1):
      #   if sample_amount > amount_already_produced:
      #     thread = threading.Thread(target=generate_data_entry, kwargs={"file_sample_number":f"{sample_amount}_irradiation", "max_cover_irradiation":i})
      #     threads.append(thread)
      #   sample_amount+=1
      # for i in np.arange(25, 35, 1):
      #   if sample_amount > amount_already_produced:
      #     generate_data_entry(file_sample_number=f"{sample_amount}_H2_demand", demand_H2=i)
      #     print(sample_amount)
      #   sample_amount+=1

          # print("H2 storage capacity")
          # for i in np.arange(15, 25, 1):
          #   if sample_amount > amount_already_produced:
          #     generate_data_entry(file_sample_number=f"{sample_amount}_H2_storage", capacity_H2_storage=i)
          #     print(sample_amount)
          #   sample_amount+=1

#Executions at a time
AMOUNT_THREADS = 20
beginning_index = 0
end_index = AMOUNT_THREADS

amount_of_threads = len(threads)

while end_index < amount_of_threads:
  for thread in threads[beginning_index:end_index]:
      thread.start()
      thread.join()
  beginning_index = beginning_index + AMOUNT_THREADS if beginning_index + AMOUNT_THREADS < amount_of_threads else end_index
  end_index = end_index + AMOUNT_THREADS if end_index + AMOUNT_THREADS < amount_of_threads else amount_of_threads

Streaming output truncated to the last 5000 lines.
best_combination: [500000000, 1100000000, 100000000, 860000000], minimum temp: -2454399429.016678
14841_season
best_combination: [500000000, 1100000000, 100000000, 860000000], minimum temp: -1649336519.241981
14842_battery_power
best_combination: [500000000, 1100000000, 100000000, 860000000], minimum temp: -3820354807.0234394
14843_season
best_combination: [500000000, 1080000000, 100000000, 840000000], minimum temp: -3292799344.2346187
14844_season
best_combination: [500000000, 1100000000, 100000000, 860000000], minimum temp: -2886043777.8151016
14845_season
best_combination: [500000000, 1100000000, 100000000, 860000000], minimum temp: -2464176969.0016565
14846_season
best_combination: [500000000, 1100000000, 100000000, 860000000], minimum temp: -1926129422.0004978
14847_battery_power
best_combination: [500000000, 1100000000, 100000000, 860000000], minimum temp: -3804354570.851684
14848_season
best_combination: [500000000, 1100000000, 

In [14]:
import os
len(os.listdir("dataset_reduced"))

17340

In [15]:
import shutil
shutil.make_archive("dataset_v7", 'zip', "dataset_reduced")

'/content/dataset_v7.zip'

In [ ]:
from google.colab import files
files.download('dataset_v4.zip')

In [ ]:
import os
import shutil
for file_name in os.listdir():
  if "json" in file_name:
    os.remove(file_name)